# Step 3: Model Extensions - Lightweight Fine-tuning

This notebook fine-tunes a DeBERTa-small model using LoRA (Low-Rank Adaptation) for the preference prediction task.

## 1. Install Libraries

In [ ]:
!pip install -q transformers peft accelerate bitsandbytes torch

## 2. Load and Prepare Data

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig

# Load data
train_df = pd.read_csv('dataset/train.csv')
test_df = pd.read_csv('dataset/test.csv')
submission_df = pd.read_csv('dataset/sample_submission.csv')

# Prepare target variable
conditions = [train_df['winner_model_a'] == 1, train_df['winner_model_b'] == 1, train_df['winner_tie'] == 1]
choices = [0, 1, 2] # 0: model_a, 1: model_b, 2: tie
train_df['label'] = np.select(conditions, choices, default=-1)
train_df = train_df[train_df['label'] != -1].reset_index(drop=True)

# Create combined text field
def create_text(row):
    return f"""prompt: {row['prompt']}

response_a: {row['response_a']}

response_b: {row['response_b']}"""

train_df['text'] = train_df.apply(create_text, axis=1)
test_df['text'] = test_df.apply(create_text, axis=1)

# Split train data for validation
train_texts, val_texts, train_labels, val_labels = train_test_split(train_df['text'], train_df['label'], test_size=0.1, random_state=42)

## 3. Tokenization

In [ ]:
model_name = 'microsoft/deberta-v3-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_df['text'].tolist(), truncation=True, padding=True, max_length=512)

class PreferenceDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = PreferenceDataset(train_encodings, train_labels.tolist())
val_dataset = PreferenceDataset(val_encodings, val_labels.tolist())

## 4. Model Setup and LoRA Configuration

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3,
    load_in_8bit=True,
    device_map='auto'
)

lora_config = LoraConfig(
    r=16, # Rank
    lora_alpha=32,
    target_modules=['query_proj', 'value_proj'],
    lora_dropout=0.05,
    bias='none',
    task_type="SEQ_CLS"
)

model = get_peft_model(model, lora_config)

## 5. Fine-tuning

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1, # A single epoch for a quick baseline
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()

## 6. Prediction and Submission

In [ ]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

test_dataset = TestDataset(test_encodings)

predictions = trainer.predict(test_dataset)
probs = torch.nn.functional.softmax(torch.from_numpy(predictions.predictions), dim=-1).numpy()

submission_df['winner_model_a'] = probs[:, 0]
submission_df['winner_model_b'] = probs[:, 1]
submission_df['winner_tie'] = probs[:, 2]

submission_df.to_csv('submission_finetuned.csv', index=False)

submission_df.head()